# Forecasting 

![extrapolating](https://imgs.xkcd.com/comics/extrapolating.png)

In this lesson, we will practice forecasting using the following methods:  

- Last observed value  
- Simple average  
- Moving average  
- Holt's Linear Trend  
- Previous cycle  


______________________________


We will walk through steps from previous lessons to get the data ready to model

- Acquire data: prepare.acquire_store_data()  
- Prepare data: prepare.prep_store_data()  
- Split data: prepare.split_store_data()  

Then we will forecast and evaluate using each method. 

## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

import env

## Acquire

We will acquire the store-item-demand data for this lesson from the sql database. 

In [2]:
# define get_connection
def get_connection (db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
# assign query to variable 
query = ' SELECT stores.*, items.*,sales.sale_date, sales.sale_amount\
            FROM sales \
            JOIN stores USING (store_id)\
            JOIN items USING (item_id);'

In [4]:
# read sql query using pd.read_sql()
df = pd.read_sql(query, get_connection('tsa_item_demand'))

In [5]:
# take a peek
df.head()


,store_id,store_address,store_zipcode,store_city,store_state,item_id,item_upc14,item_upc12,item_brand,item_name,item_price,sale_date,sale_amount
0,1,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,2013-01-01,13
1,1,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,2013-01-02,11
2,1,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,2013-01-03,14
3,1,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,2013-01-04,13
4,1,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,2013-01-05,10


## Prepare


1. sale_date to datetime
2. sort values by date
3. set index
4. new field: dollars_sold = sale_amount * item_price
5. rename sale_amount to items_sold to make the two columns easier to understand what the data represents. 
6. resample daily (The original granularity is daily, but there are multiple records of the same days across multiple stores.)
7. remove leap days!

In [6]:
# sale_date to datetime
df = df.assign(ds=pd.to_datetime(df.sale_date))

In [7]:
# sort values by date
df=df.sort_values('ds')

In [8]:
# set index
df=df.set_index('ds')

In [9]:
# create dollars_sold = sale_amount * item_price
df=df.assign(dollars_sold = df.sale_amount * df.item_price)

In [10]:
# create items_sold from sale_amount (rename)
df=df.assign(items_sold = df.sale_amount)


In [11]:
# resample daily, summing dollars_sold and items_sold
df=df.groupby(['ds'])[['dollars_sold', 'items_sold']].sum()


In [12]:
df.head(1)

,dollars_sold,items_sold
ds,,
2013-01-01,73844.01,13696


In [ ]:
df.assign(ds=pd.to_datetime(df.sale_date)).sort_values('ds').
assign(dollars_sold=df.sale_amount*df.item_price).
assign(item)

In [13]:
# sale_date to datetime
df = df.assign(ds=pd.to_datetime(df.sale_date))
ds = pd.to_datetime(df.sale_date)
# sort values by date
df.sort_values('ds')
# set index
df.set_index('ds')
# create dollars_sold = sale_amount * item_price
df.assign(dollars_sold = df.sale_amount * df.item_price)
# create items_sold from sale_amount (rename)
df.assgin(items_sold = df.sale_amount)

# resample daily, summing dollars_sold and items_sold
df.groupby(['ds'])[['dollars_sold', 'items_sold']]

# set index
df.set_index('ds')

# remove leap days

AttributeError: 'DataFrame' object has no attribute 'sale_date'

We will resample to daily, but essentially what we are doing is grouping by the day and aggregating using sum. The original granularity is daily, but there are multiple records of the same days across multiple stores. 

## Split

1. We will use the training proportion method to split.    
2. Identify the total length of the dataframe and multiple by `train_prop` to get the number of rows that equates to the first x% of the dataframe, which equates to the first x% of the time covered in the data.   (`x = train_prop * 100`)  
3. Select row indices from 0 up to the index representing x-percentile for train, and from the index representing x-percentile through the end of the dataframe for test. In both of these, we will reset the index in order to return dataframes sorted by datetime.  
4. Return train and test dataframes.  

In [ ]:
# compute num of rows that are 50% of total rows and assign to variable train_size


# compute num of rows that are 30% of total rows and assign to variable validate_size


# make test_size the number of rows remaining (test_size = total # of rows - train_size - validate_size)


# compute the row number at which the switch from validate to test happens. 


# split into train, validation, test


**Verify Splits**

Does the length of each df equate to the length of the original df? 

In [ ]:
# sum of train, validate and test = total number of rows? 


Does the first row of original df equate to the first row of train? 

In [ ]:
# test the row starts


Is the last row of train the day before the first row of validate? And the same for validate to test? 

In [ ]:
# test the split between validate and test


Is the last row of test the same as the last row of our original dataframe? 

In [ ]:
# compare the last row of test to last row of df


Let's plot our data first, viewing where the data is split into train, validate, and test. 

In [ ]:
# plot the data points, color by train, validate, test

Before we try out different methods for forecasting sales and number of items sold, let's create a couple of functions that will be helpful in evaluating each of the methods that follow. 

`evaluate()` will compute the Mean Squared Error and the Rood Mean Squared Error to evaluate.  

In [ ]:
# define evaluation function to compute rmse

def evaluate(target_var):


`plot_and_eval()` will use the evaluate function and also plot train and test values with the predicted values in order to compare performance. 

In [ ]:
# plot and evaluate: plot
def plot_and_eval(target_var):
    '''
    a function to evaluate forecasts by computing the rmse and plot train and validate along with predictions
    '''
    


Write `append_eval_df(model_type)` to append evaluation metrics for each model type, target variable, and metric type, along with the metric value into our `eval_df` data frame object. Which we will create an empty `eval_df` dataframe object to start. 

In [ ]:
# Create the empty dataframe with model_type, target_var, rmse

In [ ]:
# check out the data frame

In [ ]:
# Define function to store rmse for comparison purposes

def append_eval_df(model_type, target_var):
    '''
    
    '''
    

## Forecast 

### Last observed value

The simplest method for forecasting is to predict all future values to be the last observed value.  

**Make Predictions**

In [ ]:
# create var 'items' with last observed value

# create dollars 'items' with last observed value

In [ ]:
# make predictions by adding those values to new dataframe yhat_df


You can see, when peeking into yhat_df, that every predicted value is the same.  

**Plot Actual vs. Predicted Values**

Now, let's plot actual and predicted values

**Evaluate** 

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

### Simple Average

Take the simple average of historical values and use that value to predict future values.   

This is a good option for an initial baseline. Every future datapoint (those in 'test') will be assigned the same value, and that value will be the overall mean of the values in train. 

**Make Predictions**

**Plot Actual vs. Predicted Values**

Now, let's plot and evaluate the performance of our time series model using **Simple Average**

**Evaluate**

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

### Moving Average

In this example, we will use a 30-day moving average to forecast. In other words, the average over the last 30-days will be used as the forecasted value. 

**Make Predictions**

**Plot Actual vs. Predicted Values**

Now, let's plot and evaluate the performance of our time series model using **Moving Average**

**Evaluate**

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.colums:
    eval_df = append_eval_df(model_type = '30d moving average', target_var=col)

In [ ]:
eval_df.sort_values(by=['rmse']).groupby('target_var').first()

Let's try out several other values for periods:

In [ ]:
period = 120
items = round(train['items_sold'].rolling(period).median().iloc[-1],0)
dollars = round(train['dollars'].rolling(period).median().loc[-1],2)
yhat_df=make_predictions(items,dollars)



Which is best so far? 

In [ ]:
# get the min rmse for each variable

In [ ]:
# filter only the rows that match those rmse to find out 
# which models are best thus far


### Holt's Linear Trend

Exponential smoothing applied to both the average and the trend (slope).  

- $\alpha$ / smoothing_level: smoothing parameter for mean. Values closer to 1 will have less of a smoothing effect and will give greater weight to recent values.   
- $\beta$ / smoothing_slope: smoothing parameter for the slope. Values closer to 1 will give greater weight to recent slope/values. 


**Seasonal Decomposition**

First, let's take a look at the seasonal decomposition for each target. 

#### Basic Holt's Linear Trend

**Make Predictions**

Now, like we would when using sklearn, we will create the Holt object, fit the model, and make predictions. 

Holt: 

- exponential = True/False (exponential vs. linear growth, additive vs. multiplicative)

fit: 

- smoothing_level ($\alpha$): value between (0,1)
- smoothing_slope ($\beta$): value between (0,1)

**Plot Actual vs. Predicted Values**

**Evaluate**

### Predict Based on Previous Cycle

Take all the 2016 data points, compute the daily delta, year-over-year, average that delta over all the days, and adding that average to the previous year's value on a day will give you the forecast for that day. 

If a primary cycle is weekly, then you may want to do this on a week-over-week cadence. 

In the below example:  
1. Compute the 365 average year over year differences from 2013 through 2015
2. Add that average delta to the values during 2015. 
3. Set the index in your yhat dataframe to represent the dates those predictions are make for. 

Let's get started....

**Re-split data**

**Make Predictions**

In [ ]:
# set yhat_df to index of validate


**Plot and Evaluate**

## Conclusion

Which model did the best? 

In [ ]:
# get the min rmse for each variable


# filter only the rows that match those rmse to find out 
# which models are best thus far


Let's test it out on our out-of-sample data

We will be using train + validate to predict test. 

## Exercises

The end result of this exercise should be a Jupyter notebook named `model`.

Using [saas.csv](https://ds.codeup.com/saas.csv) or log data from API usage or store_item_sales

1. Split data (train/validate/test) and resample by any period, except daily, and aggregate using the sum. 
2. Forecast, plot and evaluate using each of the 4 parametric based methods we discussed:
    - Simple Average
    - Moving Average
    - Holt's Linear Trend Model
    - Based on previous year/month/etc., this is up to you.

Optional: Using store item demand

1. Predict 2018 total **monthly** sales for a single store and/or item by creating a model.
2. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
3. Plot the 2018 monthly sales predictions.